In [110]:
import pandas as pd
import sqlite3
from datetime import datetime

### Criando um dataframe usando o arquivo CSV

In [111]:
path_orig = '~/teste_ailos/base_files/base_originacao.csv'
path_cart = '~/teste_ailos/base_files/base_carteira.csv'

base_carteira = pd.read_csv(path_cart, sep=';', encoding='latin-1')
base_originacao = pd.read_csv(path_orig, sep=';', encoding='latin-1')

In [112]:
base_originacao

,COOPERATIVA,CPF,CONTA,CONTRATO,DSPRODUTO,DTCONTRATO,VLCONTRATO,VLPARCELA,QTPARCELAS
0,A,32,496314,677366,Financiamento,13/10/2021,"4.263,60","153,00",36.0
1,A,38,624412,644724,Empréstimo,01/09/2021,"81,60","10,20",10.0
2,A,154,49834,349384,Empréstimo,25/08/2020,"9.761,40","204,00",72.0
3,A,186,398914,444074,Empréstimo,08/01/2021,"4.314,60","163,20",36.0
4,A,430,549582,372569,Empréstimo,02/10/2020,"5.548,80","183,60",48.0
...,...,...,...,...,...,...,...,...,...
7212,E,722959,116706,109717,Financiamento,21/10/2021,"2.131,80","71,40",48.0
7213,E,724099,85550,86794,Empréstimo,24/05/2021,"1.295,40","163,20",9.0
7214,E,726648,4018,68566,Financiamento,13/01/2021,"28.998,60","459,00",120.0
7215,E,727495,4393,92571,Empréstimo,29/06/2021,"7.211,40","265,20",36.0


In [113]:
base_carteira

,DTBASE,COOPERATIVA,CPF,CONTA,CONTRATO,DSPRODUTO,DTCONTRATO,VLCONTRATO,SDDEVEDOR,QTDIASATRASO,SDATRASO,VLPROVISAO
0,31/01/2021,A,154,49834,349384,Empréstimo,25/08/2020,"9.761,40","9.557,40",0.0,"0,00","30,60"
1,31/01/2021,A,186,398914,444074,Empréstimo,08/01/2021,"4.314,60","4.365,60",0.0,"0,00","81,60"
2,31/01/2021,A,430,549582,372569,Empréstimo,02/10/2020,"5.548,80","5.273,40",0.0,"0,00","20,40"
3,31/01/2021,A,512,217818,223696,Empréstimo,31/01/2020,"3.111,00","2.539,80",0.0,"0,00","10,20"
4,31/01/2021,A,754,546819,435007,Empréstimo,23/12/2020,"601,80","601,80",19.0,"601,80","10,20"
...,...,...,...,...,...,...,...,...,...,...,...,...
44324,31/12/2021,E,722959,116706,109717,Financiamento,21/10/2021,"2.131,80","2.152,20",0.0,"0,00","10,20"
44325,31/12/2021,E,724099,85550,86794,Empréstimo,24/05/2021,"1.295,40","316,20",0.0,"0,00","10,20"
44326,31/12/2021,E,726648,4018,68566,Financiamento,13/01/2021,"28.998,60","27.917,40",0.0,"0,00","81,60"
44327,31/12/2021,E,727495,4393,92571,Empréstimo,29/06/2021,"7.211,40","3.325,20",0.0,"0,00","10,20"


### Limpando e formatando os dados da tabela

In [114]:
## transformando as colunas numéricas de str para em float 
print(type(base_originacao['VLCONTRATO'][0]))
base_originacao.VLCONTRATO = base_originacao.VLCONTRATO.str.replace('.', '').str.replace(',', '.').apply(lambda x: float(x))
base_originacao.VLPARCELA = base_originacao.VLPARCELA.str.replace('.', '').str.replace(',', '.').apply(lambda x: float(x))

base_carteira.VLCONTRATO = base_carteira.VLCONTRATO.str.replace('.', '').str.replace(',', '.').apply(lambda x: float(x))
base_carteira.SDDEVEDOR = base_carteira.SDDEVEDOR.str.replace('.', '').str.replace(',', '.').apply(lambda x: float(x))
base_carteira.SDATRASO = base_carteira.SDATRASO.str.replace('.', '').str.replace(',', '.').apply(lambda x: float(x))
base_carteira.VLPROVISAO = base_carteira.VLPROVISAO.str.replace('.', '').str.replace(',', '.').apply(lambda x: float(x))


## transformando a coluna de data em date
base_originacao.DTCONTRATO = pd.to_datetime(base_originacao.DTCONTRATO, format='%d/%m/%Y')

base_carteira.DTCONTRATO = pd.to_datetime(base_carteira.DTCONTRATO, format='%d/%m/%Y')
base_carteira.DTBASE = pd.to_datetime(base_carteira.DTBASE, format='%d/%m/%Y')


<class 'str'>


/tmp/ipykernel_85846/1279007357.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  base_originacao.VLCONTRATO = base_originacao.VLCONTRATO.str.replace('.', '').str.replace(',', '.').apply(lambda x: float(x))
/tmp/ipykernel_85846/1279007357.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  base_originacao.VLPARCELA = base_originacao.VLPARCELA.str.replace('.', '').str.replace(',', '.').apply(lambda x: float(x))
/tmp/ipykernel_85846/1279007357.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  base_carteira.VLCONTRATO =

In [115]:
base_originacao.head()

,COOPERATIVA,CPF,CONTA,CONTRATO,DSPRODUTO,DTCONTRATO,VLCONTRATO,VLPARCELA,QTPARCELAS
0,A,32,496314,677366,Financiamento,2021-10-13,4263.6,153.0,36.0
1,A,38,624412,644724,Empréstimo,2021-09-01,81.6,10.2,10.0
2,A,154,49834,349384,Empréstimo,2020-08-25,9761.4,204.0,72.0
3,A,186,398914,444074,Empréstimo,2021-01-08,4314.6,163.2,36.0
4,A,430,549582,372569,Empréstimo,2020-10-02,5548.8,183.6,48.0


In [116]:
base_carteira.head()

,DTBASE,COOPERATIVA,CPF,CONTA,CONTRATO,DSPRODUTO,DTCONTRATO,VLCONTRATO,SDDEVEDOR,QTDIASATRASO,SDATRASO,VLPROVISAO
0,2021-01-31,A,154,49834,349384,Empréstimo,2020-08-25,9761.4,9557.4,0.0,0.0,30.6
1,2021-01-31,A,186,398914,444074,Empréstimo,2021-01-08,4314.6,4365.6,0.0,0.0,81.6
2,2021-01-31,A,430,549582,372569,Empréstimo,2020-10-02,5548.8,5273.4,0.0,0.0,20.4
3,2021-01-31,A,512,217818,223696,Empréstimo,2020-01-31,3111.0,2539.8,0.0,0.0,10.2
4,2021-01-31,A,754,546819,435007,Empréstimo,2020-12-23,601.8,601.8,19.0,601.8,10.2


### Criando um DB Sqlite para fazer consultas em SQL

In [117]:
%reload_ext sql
%sql sqlite:///teste_ailos.db

In [118]:
## criando tabela no db para consulta em SQL
conn = sqlite3.connect('teste_ailos.db')
cur = conn.cursor()
cur.execute('DROP TABLE IF EXISTS base_originacao')
cur.execute('DROP TABLE IF EXISTS base_carteira')
conn.commit()

base_originacao.to_sql('base_originacao', conn)
base_carteira.to_sql('base_carteira', conn)


44329

# RESOLUÇÃO DO EXERCÍCIO EM SQL

### A) Calcule a quantidade de dias distintos DTCONTRATO e a soma dos VLCONTRATO dos contratos originados

In [119]:
%%sql
select * from base_originacao limit 5;

 * sqlite:///teste_ailos.db
Done.


index,COOPERATIVA,CPF,CONTA,CONTRATO,DSPRODUTO,DTCONTRATO,VLCONTRATO,VLPARCELA,QTPARCELAS
0,A,32,496314,677366,Financiamento,2021-10-13 00:00:00,4263.6,153.0,36.0
1,A,38,624412,644724,Empréstimo,2021-09-01 00:00:00,81.6,10.2,10.0
2,A,154,49834,349384,Empréstimo,2020-08-25 00:00:00,9761.4,204.0,72.0
3,A,186,398914,444074,Empréstimo,2021-01-08 00:00:00,4314.6,163.2,36.0
4,A,430,549582,372569,Empréstimo,2020-10-02 00:00:00,5548.8,183.6,48.0


In [120]:
%%sql
select * from base_carteira limit 5;

 * sqlite:///teste_ailos.db
Done.


index,DTBASE,COOPERATIVA,CPF,CONTA,CONTRATO,DSPRODUTO,DTCONTRATO,VLCONTRATO,SDDEVEDOR,QTDIASATRASO,SDATRASO,VLPROVISAO
0,2021-01-31 00:00:00,A,154,49834,349384,Empréstimo,2020-08-25 00:00:00,9761.4,9557.4,0.0,0.0,30.6
1,2021-01-31 00:00:00,A,186,398914,444074,Empréstimo,2021-01-08 00:00:00,4314.6,4365.6,0.0,0.0,81.6
2,2021-01-31 00:00:00,A,430,549582,372569,Empréstimo,2020-10-02 00:00:00,5548.8,5273.4,0.0,0.0,20.4
3,2021-01-31 00:00:00,A,512,217818,223696,Empréstimo,2020-01-31 00:00:00,3111.0,2539.8,0.0,0.0,10.2
4,2021-01-31 00:00:00,A,754,546819,435007,Empréstimo,2020-12-23 00:00:00,601.8,601.8,19.0,601.8,10.2


In [121]:
%%sql
select 
    count(distinct DTCONTRATO) as qnt_dias, round(sum(VLCONTRATO)) as sum_contrato
from
    base_originacao;

 * sqlite:///teste_ailos.db
Done.


qnt_dias,sum_contrato
921,32816287.0


### B) Para DTBASE = 31/12/2021 da base "base_carteira.csv", faça a soma dos SDDEVEDOR e a média simples de QTDIASATRASO

In [122]:
%%sql
select 
    sum(SDDEVEDOR) as sum_saldo_dev, avg(QTDIASATRASO) as avg_dias_atraso
from 
    base_carteira
where
    DTBASE >= '2021-12-31';

 * sqlite:///teste_ailos.db
Done.


sum_saldo_dev,avg_dias_atraso
18183172.800000023,31.896417781614158


# RESOLUÇÃO DO EXERCÍCIO EM PYTHON

### A) Calcule a quantidade de dias distintos DTCONTRATO e a soma dos VLCONTRATO dos contratos originados

In [123]:
base_originacao.head()

,COOPERATIVA,CPF,CONTA,CONTRATO,DSPRODUTO,DTCONTRATO,VLCONTRATO,VLPARCELA,QTPARCELAS
0,A,32,496314,677366,Financiamento,2021-10-13,4263.6,153.0,36.0
1,A,38,624412,644724,Empréstimo,2021-09-01,81.6,10.2,10.0
2,A,154,49834,349384,Empréstimo,2020-08-25,9761.4,204.0,72.0
3,A,186,398914,444074,Empréstimo,2021-01-08,4314.6,163.2,36.0
4,A,430,549582,372569,Empréstimo,2020-10-02,5548.8,183.6,48.0


In [124]:
Qnt_dias = base_originacao.DTCONTRATO.unique().shape[0]
Sum_contrato = round(base_originacao.VLCONTRATO.sum(), 2)

print('A quantidade de dias distintos é: ', Qnt_dias)
print('A soma dos contratos é: ', Sum_contrato)

A quantidade de dias distintos é:  921
A soma dos contratos é:  32816286.6


### B) Para DTBASE = 31/12/2021 da base "base_carteira.csv", faça a soma dos SDDEVEDOR e a média simples de QTDIASATRASO

In [125]:
base_carteira.head()

sum_saldo_dev = base_carteira[base_carteira.DTBASE >= '2021-12-31'].SDDEVEDOR.sum()
avg_dias_atraso = base_carteira[base_carteira.DTBASE >= '2021-12-31'].QTDIASATRASO.mean()

print('A soma do saldo devedor é: ', sum_saldo_dev)
print('A média dos dias de atraso é: ', avg_dias_atraso)

A soma do saldo devedor é:  18183172.799999997
A média dos dias de atraso é:  31.896417781614158
